In [23]:
import pandas as pd

df = pd.read_json('./Data/embeddings/2019_embedding.json')

In [24]:
df

,uri,dataType,title,dateTimePub,body,url,vector
0,1278241173,news,Los números y desafíos que dejaron las Fiestas...,2019-09-29 19:26:00+00:00,Los números y desafíos que dejaron las Fiestas...,https://www.tvn.cl/programas/sincorbata/capitu...,"[0.0332329348, 0.0892551243, 0.0313695408, 0.0..."
1,1278241173,news,Los números y desafíos que dejaron las Fiestas...,2019-09-29 19:26:00+00:00,"""Periodista Destacado"" Sin Corbata Capítulo 09...",https://www.tvn.cl/programas/sincorbata/capitu...,"[0.0722330883, 0.25242516400000004, -0.1440636..."
2,1278241174,news,Los desafíos viales que tuvo el fin de semana ...,2019-09-29 19:26:00+00:00,Los desafíos viales que tuvo el fin de semana ...,https://www.tvn.cl/programas/sincorbata/destac...,"[-0.0405375101, -0.1526000798, 0.1178539991000..."
3,1278241174,news,Los desafíos viales que tuvo el fin de semana ...,2019-09-29 19:26:00+00:00,.11.2018 ¿Chile está al debe en innovación? Si...,https://www.tvn.cl/programas/sincorbata/destac...,"[-0.31152355670000004, -0.3042310178, 0.011176..."
4,1281140962,news,Capítulo 1 de octubre: Abuela quiere el cuidad...,2019-10-01 22:10:00+00:00,Capítulo 1 de octubre: Abuela quiere el cuidad...,https://www.tvn.cl/programas/carmengloria/capi...,"[0.10201750700000001, 0.0172973499, 0.11956939..."
...,...,...,...,...,...,...,...
69231,1366546580,news,Serviu tramitará pagos de remuneraciones a ...,2019-12-06 20:44:00+00:00,"que ""desde el principio hemos tenido una comun...",https://www.cooperativa.cl/noticias/pais/regio...,"[0.070639953, 0.12183474, -0.0825189576, -0.04..."
69232,1366546580,news,Serviu tramitará pagos de remuneraciones a ...,2019-12-06 20:44:00+00:00,de los trabajadores directos de los proyectos ...,https://www.cooperativa.cl/noticias/pais/regio...,"[-0.1907450557, 0.1127685085, -0.0773107409000..."
69233,1366648501,news,Histórico presupuesto del Minvu en Ñuble: S...,2019-12-06 22:52:00+00:00,Histórico presupuesto del Minvu en Ñuble: Se q...,https://www.cooperativa.cl/noticias/pais/regio...,"[0.1351340413, 0.186160773, 0.0454363413, 0.09..."
69234,1366648501,news,Histórico presupuesto del Minvu en Ñuble: S...,2019-12-06 22:52:00+00:00,ados cuando aún se dependía territorialmente d...,https://www.cooperativa.cl/noticias/pais/regio...,"[0.17274466160000002, 0.032037999500000004, -0..."


In [20]:
a = df.sample(3)


In [2]:
# Convertir el DataFrame filtrado a una lista de diccionarios
data_list = a.to_dict(orient='records')

# Crear el diccionario final con la clave 'rows'
final_structure = {'rows': data_list}
#final_df = pd.DataFrame(final_structure)
#final_df.to_json('./Data/embeddings/example_embedding.json', orient='records', force_ascii=False)
ruta_archivo = './Data/embeddings/embegddings_example.json'
with open(ruta_archivo, 'w', encoding='utf-8') as file:
    json.dump(final_structure, file, ensure_ascii=False, indent=None)


NameError: name 'a' is not defined

In [14]:
import os
from pymilvus import MilvusClient, connections, FieldSchema, CollectionSchema, DataType, Collection, utility

# Replace uri and token with your own
client = MilvusClient(
    uri=os.getenv("MILVUS_ENDPOINT"),
    token=os.getenv("MILVUS_API_KEY") # API key or a colon-separated cluster username and password
)

# Connect to cluster
connections.connect(
  alias='default',
  # Public endpoint obtained from Zilliz Cloud
  uri=os.getenv("MILVUS_ENDPOINT"),
  # API key or a colon-separated cluster username and password
  token=os.getenv("MILVUS_API_KEY")
)

In [37]:
# 1. Define fields
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="dataType", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="uri", dtype=DataType.INT64, max_length=1024),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=2048),
    FieldSchema(name="body", dtype=DataType.VARCHAR, max_length=4096),
    FieldSchema(name="dateTimePub", dtype=DataType.VARCHAR, max_length=1024),  # Fecha de publicación
    FieldSchema(name="url",dtype=DataType.VARCHAR, max_length = 1024),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=384),
]
#uri	dataType	title	dateTimePub	body	url	vector

In [38]:
# 2. Build the schema
schema = CollectionSchema(
    fields,
    description="Schema of news",
    enable_dynamic_field=True
)

In [39]:
# 3. Create collection
collection = Collection(
    name='News_2019', 
    description="",
    schema=schema
)

In [40]:
# 4. Index collection
# 'index_type' defines the index algorithm to be used.
#    AUTOINDEX is the only option.
#
# 'metric_type' defines the way to measure the distance 
#    between vectors. Possible values are L2, IP, and Cosine,
#    and defaults to Cosine.
index_params = {
    "index_type": "AUTOINDEX",
    "metric_type": "L2",
    "params": {}
}

# To name the index, do as follows:
collection.create_index(
    field_name="vector", 
    index_params=index_params,
    index_name='vector_index'
)

Status(code=0, message=)

In [41]:
# 5. Load collection
collection.load()

# Get loading progress
progress = utility.loading_progress('News_2019')

print(progress)

{'loading_progress': '100%'}


In [29]:
# Convertir el DataFrame filtrado a una lista de diccionarios
data_list = df.to_dict(orient='records')

# Crear el diccionario final con la clave 'rows'
final_structure = {'rows': data_list}
#final_df = pd.DataFrame(final_structure)
#final_df.to_json('./Data/embeddings/example_embedding.json', orient='records', force_ascii=False)
ruta_archivo = './Data/embeddings/2019_embedding_upload.json'
with open(ruta_archivo, 'w', encoding='utf-8') as file:
    json.dump(final_structure, file, ensure_ascii=False, indent=None)


In [21]:
import json
archivo_entrada = './Data/embeddings/2019_embedding_upload.json'
# Leer el JSON original desde el archivo
with open(archivo_entrada, 'r') as f:
    data = json.load(f)


In [48]:
res = client.insert(
  collection_name='News_2019',
  data=data['rows'][60000:]
)